<h1 id="tocheading">Table of Contents</h1>
<div id="toc"></div>

In [1]:
%%javascript
$.getScript('https://kmahelona.github.io/ipython_notebook_goodies/ipython_notebook_toc.js')

In [2]:
# Import some necessary modules
%matplotlib inline
import numpy as np
import scipy.stats as stats
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib as mpl
import pandas as pd
import time
import scipy as sp
import pickle
%load_ext autoreload
%autoreload 2

In [3]:
# Import MuSiCal
import musical

# Overview 

In this notebook, we demonstrate how to refit an input matrix $X$ against a signature catalog. Refitting can be performed as a standalone task for predicting signature exposures, or as a downstream step after *de novo* signature discovery and matching the *de novo* signatures to the catalog. 

# Input data

The input data for refitting is the mutation count matrix $X$ and the signature catalog $W$. 

## The mutation count matrix

Here we use a simulated dataset based on PCAWG breast cancers to demonstrate how to perform refitting. The dataset contains 8 SBS signatures (SBS1, 2, 3, 5, 8, 13, 17b, and 18). Note that some minor signatures in the original PCAWG breast cancer dataset are removed for simplicity.  

Below, `X` is the simulated mutation count matrix. `W_true` is the true signatures present in the dataset (i.e., the 8 SBS signatures). `H_true` is the true exposure matrix from which `X` is simulated. 

In reality, only `X` is needed, since `W_true` and `H_true` are unknown. We read the truth information here so that we can evaluate the refitting results.

In [4]:
X = pd.read_csv('./data/simulated/data_simul.pcawg_breast_8sig.X.csv', index_col=0)
W_true = pd.read_csv('./data/simulated/data_simul.pcawg_breast_8sig.W_true.csv', index_col=0)
H_true = pd.read_csv('./data/simulated/data_simul.pcawg_breast_8sig.H_true.csv', index_col=0)

In [5]:
X.head()

,SP117956,SP117402,SP117378,SP117244,SP117079,SP117312,SP117108,SP117370,SP117850,SP117593,...,SP7291,SP7171,SP10563,SP12186,SP10944,SP6673,SP118012,SP124199,SP116365,SP2159
Type,,,,,,,,,,,,,,,,,,,,,
A[C>A]A,20,25,18,13,17,22,36,21,15,13,...,51,55,52,60,100,389,21,32,78,14
A[C>A]C,13,23,19,9,12,13,16,18,6,8,...,34,21,31,28,86,324,13,27,65,20
A[C>A]G,5,6,4,2,1,1,2,2,0,2,...,5,5,5,7,10,35,4,2,5,5
A[C>A]T,17,13,8,11,5,9,26,10,7,8,...,22,24,22,36,67,312,14,17,41,7
C[C>A]A,14,19,17,17,5,8,52,22,5,6,...,54,55,45,77,110,317,26,28,76,22


In [6]:
W_true.head()

,SBS1,SBS2,SBS3,SBS5,SBS8,SBS13,SBS17b,SBS18
Type,,,,,,,,
A[C>A]A,0.000886,5.800168e-07,0.020808,0.011998,0.044098,0.001820,0.000608,0.051534
A[C>A]C,0.002280,1.480043e-04,0.016507,0.009438,0.047798,0.000721,0.000129,0.015810
A[C>A]G,0.000177,5.230151e-05,0.001751,0.001850,0.004620,0.000264,0.000058,0.002432
A[C>A]T,0.001280,9.780282e-05,0.012205,0.006609,0.046998,0.000348,0.000456,0.021414
C[C>A]A,0.000312,2.080060e-04,0.022509,0.007429,0.040098,0.001400,0.000271,0.074049


In [7]:
H_true.head()

,SP117956,SP117402,SP117378,SP117244,SP117079,SP117312,SP117108,SP117370,SP117850,SP117593,...,SP7291,SP7171,SP10563,SP12186,SP10944,SP6673,SP118012,SP124199,SP116365,SP2159
SBS1,224,320,196,169,197,354,323,327,327,359,...,336,313,567,671,1089,0,256.000000,442.000000,394.316864,481.147835
SBS2,41,796,0,638,1126,198,0,115,328,35,...,84,480,1379,2883,3403,3888,175.599823,465.865977,401.000000,2650.000000
SBS3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,3245,5526,0.000000,0.000000,3169.000000,0.000000
SBS5,1452,1634,1499,749,1051,1131,1017,945,1085,1180,...,1524,2056,2069,1783,3102,4308,1199.000000,2232.000000,1409.000000,1678.000000
SBS8,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,4430,0.000000,0.000000,0.000000,0.000000


## The signature catalog 

MuSiCal provides several signature catalogs, listed below. 

In [8]:
musical.catalog.CATALOG_NAMES

['COSMIC_v2_SBS_WGS',
 'COSMIC_v3_SBS_WGS',
 'COSMIC_v3_SBS_WES',
 'COSMIC_v3p1_SBS_WGS',
 'COSMIC_v3p2_SBS_WGS',
 'COSMIC-MuSiCal_v3p2_SBS_WGS',
 'COSMIC_v3p1_Indel',
 'MuSiCal_v4_Indel_WGS']

Let's first load the default SBS catalog, which is `COSMIC-MuSiCal_v3p2_SBS_WGS`. This catalog includes 78 COSMIC v3.2 SBS signatures and 6 SBS signatures additionally discovered by MuSiCal from PCAWG samples. Below, `catalog` is a catalog class object. Signatures in the catalog can be accessed through `catalog.W`. We see that there are in total 84 signatures. 

In [9]:
catalog = musical.load_catalog()
print(catalog.W.shape[1])

84


Other catalogs can be loaded if a name is specified. For example, the following line loads the preferred indel signature catalog. 
```
catalog = musical.load_catalog('MuSiCal_v4_Indel_WGS')
```

Directly refitting our dataset `X` against all 96 signatures in the catalog will introduce many false positives, leading to over-assignment. It is thus better to restrict our catalog to only those signatures found in the specific tumor type. 

You can select your own preferred set of signatures. But MuSiCal provides such information based on our PCAWG reanalysis. 

Below, we restrict our catalog to Breast.AdenoCA. Now, only 13 signatures remain in the catalog.  

In [10]:
catalog.restrict_catalog(tumor_type='Breast.AdenoCA')
print(catalog.W.shape[1])

13


A list of available tumor types are shown below. 

In [11]:
print(catalog.show_tumor_type_options().tolist())

['Biliary.AdenoCA', 'Bladder.TCC', 'Bone.Benign', 'Bone.Epith', 'Bone.Osteosarc', 'Breast.AdenoCA', 'Breast.DCIS', 'Breast.LobularCA', 'CNS.GBM', 'CNS.Medullo', 'CNS.Oligo', 'CNS.PiloAstro', 'Cervix.AdenoCA', 'Cervix.SCC', 'ColoRect.AdenoCA', 'Eso.AdenoCA', 'Head.SCC', 'Kidney.ChRCC', 'Kidney.RCC', 'Liver.HCC', 'Lung.AdenoCA', 'Lung.SCC', 'Lymph.BNHL', 'Lymph.CLL', 'Myeloid.AML', 'Myeloid.MDS', 'Myeloid.MPN', 'Ovary.AdenoCA', 'Panc.AdenoCA', 'Panc.Endocrine', 'Prost.AdenoCA', 'Skin.Melanoma', 'SoftTissue.Leiomyo', 'SoftTissue.Liposarc', 'Stomach.AdenoCA', 'Thy.AdenoCA', 'Uterus.AdenoCA']


We can further restrict our catalog by removing signatures associated with mismatch repair deficiency (MMRD) or polymerase proofreading deficiency (PPD) (e.g., samples with POLE-exo mutations), since we know that this simulated dataset does not contain MMRD or PPD samples. 

If you are not sure whether your dataset contains MMRD/PPD samples, you can first perform a refitting including the MMRD/PPD signatures, and then use the `musical.preprocessing` module to determine if there is a cluster of MMRD/PPD samples within your dataset. If so, you can separate these samples and perform refitting again for the two clusters of samples separately. Of course other methods can be used to determine MMRD/PPD samples, e.g., by looking for hypermutations, inspecting POLE-exo mutations, detecting microsatellite instabilities, etc. 

In this case, no additional signatures are removed, since none of the 13 breast-specific signatures are associated with MMRD or PPD. 

In [12]:
catalog.restrict_catalog(tumor_type='Breast.AdenoCA', is_MMRD=False, is_PPD=False)
print(catalog.W.shape[1])

13


We can finally obtain signatures in the catalog. 

In [13]:
W = catalog.W
print(W.columns.tolist())

['SBS1', 'SBS2', 'SBS3', 'SBS5', 'SBS8', 'SBS13', 'SBS17a', 'SBS17b', 'SBS18', 'SBS34', 'SBS41', 'SBS85', 'SBS100']


# Refitting 

Refitting can be performed with `musical.SparseNNLS`.

## Naive NNLS 

Let's first try naive NNLS. This can be achieved by setting `method` to `thresh_naive` and `thresh1` to `0`. The method `thresh_naive` simply performs NNLS, and then set signatures with relative exposures smaller than `thresh1` to have zero exposures.  

In [14]:
model = musical.SparseNNLS(method='thresh_naive', thresh1=0)
model.fit(X, W)

The resulting exposure matrix can then be obtained via `model.H`.

In [15]:
model.H.head()

,SP117956,SP117402,SP117378,SP117244,SP117079,SP117312,SP117108,SP117370,SP117850,SP117593,...,SP7291,SP7171,SP10563,SP12186,SP10944,SP6673,SP118012,SP124199,SP116365,SP2159
SBS1,215.725723,326.368392,176.571488,156.774433,235.668611,366.522626,330.854505,307.069205,322.116106,369.448259,...,315.938350,280.925789,551.549032,683.107865,1031.817207,32.066971,242.065929,466.741594,390.834755,487.062277
SBS2,48.095435,717.736836,3.023016,660.589665,1130.039751,166.450301,16.689622,137.370047,358.945498,29.233500,...,93.176691,528.630936,1364.214132,2885.365389,3543.940910,3859.651861,164.181522,486.639706,403.570479,2684.834771
SBS3,0.000000,0.000000,0.472962,0.000000,0.000000,31.424048,65.900505,1.987732,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,3192.287256,5678.834398,26.787520,0.000000,2913.017332,0.000000
SBS5,1470.298457,1666.628152,1502.627835,771.903194,1014.513359,1066.033016,819.407744,917.564784,1091.272137,1190.005305,...,1534.556389,2057.777952,2132.122638,1662.880907,2913.377932,4213.626525,1207.822115,2167.345303,1604.143052,1552.511083
SBS8,10.328492,30.905100,9.600728,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,26.944969,0.000000,0.000000,0.000000,122.545513,4363.085067,0.000000,0.000000,22.101555,0.000000


We can compare the obtained exposure matrix with the true one to evaluate the refitting result with naive NNLS. To do that, let's first reindex `H_true` so that it has the same signatures as in `model.H`. 

In [16]:
H_true_reindexed = H_true.reindex(model.H.index).fillna(0)

Then, we calculate some statistics by comparing nonzero entries in `H_true_reindexed` and those in `model.H`

In [17]:
TP = np.logical_and(H_true_reindexed > 0, model.H > 0).sum().sum()
FP = np.logical_and(H_true_reindexed == 0, model.H > 0).sum().sum()
TN = np.logical_and(H_true_reindexed == 0, model.H == 0).sum().sum()
FN = np.logical_and(H_true_reindexed > 0, model.H == 0).sum().sum()
P = TP + FN
N = TN + FP
print(TP, FP, TN, FN, P, N)

972 601 1000 1 973 1601


In [18]:
print('Sensitivity = %.3g' % (TP/P))
print('False positive rate = %.3g' % (FP/N))

Sensitivity = 0.999
False positive rate = 0.375


We see that naive NNLS leads to a high false positive rate, i.e., over-assignment. 

## Likelihood-based sparse NNLS 

MuSiCal implements a likelihood-based sparse NNLS for refitting. It can be achieved by setting `method` to `likelihood_bidirectional` in `musical.SparseNNLS`. The small nonnegative likelihood threshold `thresh1` controls the sparsity level. When `thresh1` is 0, the result is almost equivalent to naive NNLS. Stronger sparsity will be induced when `thresh1` is larger. 

In the full pipeline including *de novo* signature discovery followed by matching and refitting, this likelihood threshold will be automatically chosen by the *in silico* validation module. 

Here, let's use a reasonable threshold 0.001. 

In [19]:
model = musical.SparseNNLS(method='likelihood_bidirectional', thresh1=0.001)
model.fit(X, W)

In [20]:
model.H.head()

,SP117956,SP117402,SP117378,SP117244,SP117079,SP117312,SP117108,SP117370,SP117850,SP117593,...,SP7291,SP7171,SP10563,SP12186,SP10944,SP6673,SP118012,SP124199,SP116365,SP2159
SBS1,215.661266,325.669303,175.872104,156.234461,235.587259,365.690288,331.234329,306.740894,321.645670,369.460609,...,314.656031,280.497345,551.549032,682.556261,1033.853054,0.000000,240.965152,464.779522,390.340174,484.176909
SBS2,47.868984,717.239114,0.000000,661.147193,1129.882934,165.490219,18.605649,136.633595,360.652671,28.705664,...,93.574489,529.471274,1364.214132,2884.409335,3544.846685,3860.285108,164.158870,487.350435,403.402718,2684.789954
SBS3,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,98.639600,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,3366.398807,5648.032123,0.000000,0.000000,2957.535112,0.000000
SBS5,1478.696186,1703.770139,1524.889234,785.808682,1020.451923,1113.493340,840.451714,937.412869,1107.448645,1223.388672,...,1558.273432,2072.365755,2132.122638,1724.551709,2863.833900,4280.832566,1235.186602,2224.753489,1613.464299,1625.385047
SBS8,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,4381.726549,0.000000,0.000000,0.000000,0.000000


Again, we can compare to `H_true` to evaluate the result.

In [21]:
H_true_reindexed = H_true.reindex(model.H.index).fillna(0)

In [22]:
TP = np.logical_and(H_true_reindexed > 0, model.H > 0).sum().sum()
FP = np.logical_and(H_true_reindexed == 0, model.H > 0).sum().sum()
TN = np.logical_and(H_true_reindexed == 0, model.H == 0).sum().sum()
FN = np.logical_and(H_true_reindexed > 0, model.H == 0).sum().sum()
P = TP + FN
N = TN + FP
print(TP, FP, TN, FN, P, N)

967 13 1588 6 973 1601


In [23]:
print('Sensitivity = %.3g' % (TP/P))
print('False positive rate = %.3g' % (FP/N))

Sensitivity = 0.994
False positive rate = 0.00812


We see that the false positive rate is dramatically reduced, while the sensitivity is still reasonably high. 

# Comments 

1. Matching *de novo* signatures to the catalog can be performed in the same way as refitting, except that in matching, `X` will be the matrix of signatures to be matched. See Methods in our paper for the reasoning behind this. 

2. The `SparseNNLS` object provides many other attributes that are convenient. For example, `model.X_reconstructed` is the reconstructed mutation count matrix. `model.cos_similarities` is the cosine similarities between original data and the reconstructed spectra. `model.H_reduced` is the reduced exposure matrix, i.e., after removing signatures in the catalog that receive zero exposures in all samples. 

3. Associated signatures (e.g., APOBEC signatures SBS2 and SBS13) can be forced to co-occur using the option `indices_associated_sigs` in `SparseNNLS`. 